In [14]:
import numpy as np
import os
import json
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Function to process JSON files and prepare data
def process_json_files(folder_path):
    X_train = []
    y_train = []
    class_name = ["SKU_1L", "SKU_4L", "SKU_10L_DTS", "SKU_10L_NDTS", "SKU_20L_DTS", "SKU_20L_NDTS"]
    
    # List all JSON files in the folder
    json_files = [f for f in os.listdir(folder_path) if f.endswith('.json')]
    skus_info_data = []
    
    for json_file in json_files:
        file_path = os.path.join(folder_path, json_file)
        
        # Read and parse JSON file
        with open(file_path, 'r') as file:
            parsed_data = json.load(file)
        
        # Extract truck dimensions and SKU quantities
        truck_dimension = {item['key']: int(item['value']) for item in parsed_data['truckDimension']}
        total_used_skus = {item['skuName']: int(item['totalUsedSkusCount']) for item in parsed_data['totalUsedSkus']}
        
        temp = []
        
        for item in parsed_data['skusInfoData']:
            sku_name = item['skuName']
            skuLocalPosition = {pos['key']: float(pos['value']) for pos in item['skuLocalPosition']}
            
            sku_info = [round(skuLocalPosition['x'], 2), round(skuLocalPosition['y'], 2), round(skuLocalPosition['z'], 2)]
            temp.append(sku_info)
        
        skus_info_data.append(temp)
        
        # Create X_train entry
        X_train.append([
            truck_dimension['Length'],
            truck_dimension['Width'],
            truck_dimension['Height'],
            total_used_skus['SKU_1L'],
            total_used_skus['SKU_4L'],
            total_used_skus['SKU_10L_DTS'],
            total_used_skus['SKU_10L_NDTS'],
            total_used_skus['SKU_20L_DTS'],
            total_used_skus['SKU_20L_NDTS']
        ])
    
    # Flatten y_train positions
    for item_list in skus_info_data:
        positions = []
        for item in item_list:
            positions.extend(item)  # item contains the position [x, y, z]
        y_train.append(positions)

    print(len(X_train), len(y_train))
    max_positions_length = max(len(position) for position in y_train)

    for positions in y_train:
        while (len(positions) < max_positions_length):
            positions.append(0)
    
    return np.array(X_train), np.array(y_train)

# Process JSON files and prepare data
folder_path = 'Json'
#process_json_files(folder_path)
X_train, y_train = process_json_files(folder_path)

# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train = scaler_X.fit_transform(X_train)
y_train = scaler_y.fit_transform(y_train)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


167 167


In [15]:
# Train the Random Forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)


RandomForestRegressor(random_state=42)

In [19]:
# Predict the positions for the test set
y_pred = rf.predict(X_test)

# Denormalize the predictions
y_pred = scaler_y.inverse_transform(y_pred)

# Evaluate the model
from sklearn.metrics import mean_absolute_error

mae = mean_absolute_error(y_test, y_pred)
print(f'Model MAE: {mae}')

# Function to structure the predictions back into the desired format
def structure_predictions(predictions, input_data):
    sku_names = ["SKU_1L", "SKU_4L", "SKU_10L_DTS", "SKU_10L_NDTS", "SKU_20L_DTS", "SKU_20L_NDTS"]
    predicted_output = {}
    position_index = 0

    for sku in sku_names:
        sku_positions = []
        for _ in range(input_data[sku]):
            position = {
                'x': predictions[position_index],
                'y': predictions[position_index + 1],
                'z': predictions[position_index + 2]
            }
            sku_positions.append(position)
            position_index += 3
        predicted_output[sku] = sku_positions

    return predicted_output

# Example input for prediction
input_data = {
    'Length': 80,
    'Width': 31,
    'Height': 20,
    'SKU_1L': 200,
    'SKU_4L': 120,
    'SKU_10L_DTS': 300,
    'SKU_10L_NDTS': 396,
    'SKU_20L_DTS': 231,
    'SKU_20L_NDTS': 165
}

# Convert input data to list
input_list = [
    input_data['Length'],
    input_data['Width'],
    input_data['Height'],
    input_data['SKU_1L'],
    input_data['SKU_4L'],
    input_data['SKU_10L_DTS'],
    input_data['SKU_10L_NDTS'],
    input_data['SKU_20L_DTS'],
    input_data['SKU_20L_NDTS']
]

# Normalize input data
input_list = scaler_X.transform([input_list])

# Make prediction
predicted_positions = rf.predict(input_list)

# Denormalize the prediction
predicted_positions = scaler_y.inverse_transform(predicted_positions)

# Structure the prediction
predicted_output = structure_predictions(predicted_positions.flatten(), input_data)

# Print the predicted output
for sku, positions in predicted_output.items():
    print(f"{sku}: {positions[:5]}")  # Print the first 5 positions for brevity


Model MAE: 14.101404474014766
SKU_1L: [{'x': 0.0, 'y': 0.0, 'z': 0.0}, {'x': 0.0, 'y': 0.0, 'z': 3.7999999999999994}, {'x': 0.0, 'y': 0.0, 'z': 7.599999999999999}, {'x': 0.0, 'y': 0.0, 'z': 11.4}, {'x': 0.0, 'y': 0.0, 'z': 15.199999999999998}]
SKU_4L: [{'x': 10.164500000000002, 'y': 12.573699999999999, 'z': 14.940900000000006}, {'x': 10.196599999999997, 'y': 12.421000000000005, 'z': 17.574400000000004}, {'x': 10.293, 'y': 11.656600000000001, 'z': 19.335599999999992}, {'x': 10.292999999999997, 'y': 11.694999999999995, 'z': 21.957199999999993}, {'x': 10.293000000000003, 'y': 11.732399999999998, 'z': 24.547599999999996}]
SKU_10L_DTS: [{'x': 18.910800000000002, 'y': 8.8337, 'z': 9.2696}, {'x': 18.942, 'y': 8.68, 'z': 12.120000000000001}, {'x': 18.942, 'y': 8.68, 'z': 15.236600000000005}, {'x': 18.942, 'y': 8.718399999999997, 'z': 18.072399999999995}, {'x': 18.942, 'y': 8.7558, 'z': 20.877000000000013}]
SKU_10L_NDTS: [{'x': 37.7521, 'y': 11.014400000000002, 'z': 15.983899999999998}, {'x': 3

# prediction 2 
